In [149]:
import numpy as np
import pandas as pd
import json
import re
import MeCab
from collections import Counter

In [2]:
def read_jasonl(filename):
    with open(filename) as f:
        return [json.loads(line.rstrip('\r\n')) for line in f.readlines()]

In [3]:
flatten = lambda l: [item for sublist in l for item in sublist if len(item) is not 0]

In [4]:
with open("../data/compound_train.json", 'r') as f:
    train = json.load(f)

In [5]:
wiki_data = read_jasonl("../data/jawiki-cirrussearch-dump_of_Compound.jsonl")

## カテゴリリスト作成

In [71]:
category_list = flatten([entry['category'] for entry in wiki_data])

In [86]:
pd.DataFrame(Counter(category_list).most_common()).rename(columns={0: "category", 1: "count_"}).to_csv("../data/category_list.csv", index=False)

In [6]:
Youto_dict = {}
for entry in train['entry']:
    Youto_dict[str(entry['WikipediaID'])] = entry['Attributes']['用途']

In [7]:
train_entry = [entry for entry in wiki_data for train_entry in train['entry'] if entry['index']['_id'] == str(train_entry['WikipediaID'])]

In [13]:
pd.DataFrame(
    {"category": list(set([category for t in train_entry for category in t['category'] if is_contain(category, Youto_dict[t['index']['_id']])]))}
).to_csv("../data/category_in_train_data.csv", index=False)

In [10]:
def is_contain(category, youto_list):
    youto = ','.join(youto_list)
    
    if re.search(category, youto):
        return True
    else:
        return False

## 名詞を抽出したい

In [11]:
category_in_train_list = pd.read_csv("../data/category_in_train_data.csv")

In [30]:
article_df = pd.DataFrame()

for article in wiki_data:
    clean_text = re.sub(r'\^\s[^\^]+', '', article['text'])
    clean_text = re.sub(r'\\[a-zA-Z0-9]+', '', clean_text)
    clean_text = re.sub(r'{.+}|\"', '', clean_text)
    clean_text = re.sub(r'\s{2,}', ' ', clean_text)
    sentences_df = pd.DataFrame(
                        {
                            "sentence": list(map(lambda s: s.strip(), re.findall(".*?。", clean_text)))
                            , "_id": article['index']['_id']
                        }
                    )
    article_df = article_df.append(sentences_df)

article_df.reset_index(drop=True, inplace=True)

In [66]:
article_df.loc[
    article_df.sentence.str.contains('|'.join(category_in_train_list.category))
    & article_df.sentence.str.contains('|'.join(['用い', '利用', '使わ', '使用']))
]

,sentence,_id
25,ハロゲン化物消火設備の消火剤に使われる。,2662912
26,これは、酸素を遮断する効果に加え、ハロンの熱分解で生じたハロゲン原子が、燃焼により発生する高...,2662912
67,ロジンのままでは滑り止め（野球のロジンバッグ、ダンス会場などでの床面への粉末散布、バイオリン...,472047
72,中国では、旧時燃料や着火剤としても使われた。,472047
76,なお、ロジンを蒸留する際に残るピッチや低沸点の樹脂はバイオマス燃料として利用されている。,472047
100,過酸化物や硝酸塩、塩素酸塩などの酸化剤と混合したものは雷管に装填するのに使われたことがある。,652294
145,食品添加物として使用される。,161158
169,主に都市ガスや代燃車の燃料として使用された。,300714
170,また燃料以外にメタノールなどの合成化学原料にも用いられる。,300714
201,純粋な化合物、もしくはシソから得られるペリルアルデヒドを豊富に含む揮発性のオイルは、香り付け...,699433


In [49]:
Youto_list = flatten([article['Attributes']['用途'] for article in train['entry']])

In [67]:
article_df.loc[
    article_df.sentence.str.contains('|'.join(Youto_list).replace('（', '(').replace('）', ')'))
    & article_df.sentence.str.contains('|'.join(['用い', '利用', '使わ', '使用']))
].to_csv("../data/tmp_Youto_sentences.csv", index=False)

/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [68]:
Youto_sentence_df = pd.read_csv("../data/tmp_Youto_sentences.csv")
Youto_sentence_df.head()

,sentence,_id
0,ハロゲン化物消火設備の消火剤に使われる。,2662912
1,これは、酸素を遮断する効果に加え、ハロンの熱分解で生じたハロゲン原子が、燃焼により発生する高...,2662912
2,また、植物の呼吸キナーゼを抑制する効果があり、ポストハーベスト農薬として使用できる。,1540217
3,ロジンのままでは滑り止め（野球のロジンバッグ、ダンス会場などでの床面への粉末散布、バイオリン...,472047
4,中国では、旧時燃料や着火剤としても使われた。,472047


In [74]:
Youto_sentence_df.loc[
    Youto_sentence_df.sentence.str.contains('|'.join(category_list))
]

/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


,sentence,_id
0,ハロゲン化物消火設備の消火剤に使われる。,2662912
1,これは、酸素を遮断する効果に加え、ハロンの熱分解で生じたハロゲン原子が、燃焼により発生する高...,2662912
2,また、植物の呼吸キナーゼを抑制する効果があり、ポストハーベスト農薬として使用できる。,1540217
3,ロジンのままでは滑り止め（野球のロジンバッグ、ダンス会場などでの床面への粉末散布、バイオリン...,472047
4,中国では、旧時燃料や着火剤としても使われた。,472047
5,印刷インキ、塗料、顔料の表面コーティング、フラックスなどの用途の一部には水素を添加して脱色し...,472047
6,なお、ロジンを蒸留する際に残るピッチや低沸点の樹脂はバイオマス燃料として利用されている。,472047
7,過酸化物や硝酸塩、塩素酸塩などの酸化剤と混合したものは雷管に装填するのに使われたことがある。,652294
8,アルカリ性溶液中で銀や銅などの重金属イオンを還元するため、この反応を利用したトレンス試薬（銀...,521686
9,食品添加物として使用される。,161158


In [161]:
sentiment_df = pd.read_csv("../data/pn.csv.m3.120408.tsv"
                           , delimiter='\t'
                           , names=['word', 'sentiment', 'details']
                          )
sentiment_df = sentiment_df.loc[(sentiment_df.sentiment == 'p') | (sentiment_df.sentiment == 'n')]
sentiment_df.head()

,word,sentiment,details
20,１位,p,〜である・になる（状態）客観
25,１周年記念,p,〜である・になる（状態）客観
27,１勝,p,〜する（出来事）
32,１番,p,〜である・になる（状態）客観
46,２次感染,n,〜する（出来事）


In [165]:
sentiment_df = pd.read_csv("../data/pn_ja.dic"
                           , delimiter=':'
                           , names=['word', 'yomi', 'hinshi', 'sentiment']
                          )
sentiment_df.head()

,word,yomi,hinshi,sentiment
0,優れる,すぐれる,動詞,1.000000
1,良い,よい,形容詞,0.999995
2,喜ぶ,よろこぶ,動詞,0.999979
3,褒める,ほめる,動詞,0.999979
4,めでたい,めでたい,形容詞,0.999645


In [170]:
tmp_df = pd.read_csv("../data/train_Youto_sentence_and_heading.csv").assign(sentiment=0)
for i, (word, _, _, sentiment) in sentiment_df.iterrows():
    #tmp_df.loc[tmp_df.sentence.str.contains(word), sentiment] += 1
    tmp_df.loc[tmp_df.sentence.str.contains(word), 'sentiment'] += sentiment

In [173]:
tmp_df.sort_values('sentiment', ascending=True)

,_id,sentence,label,heading,sentiment
994,271215,燃料 ろうそくの原料 蝋紙の原料 着火剤 火吹き 固定 マッチ軸木の含浸材（燃焼材として） ...,1,用途例,-187.884752
195,3437060,日本の添付文書で重大な副作用とされているものは、 出血（脳出血、消化管出血等）、白血球症（頭...,0,副作用,-116.494045
194,3437060,米国の添付文書で頻度別に記載されている副作用は、 ・10%超に発生する副作用（太字は国内治験...,0,副作用,-108.706151
5743,2170531,重大な副作用として知られているものには、 再生不良性貧血、巨赤芽球性貧血、メトヘモグロビン血...,0,副作用,-85.464391
5313,1371546,製剤成分に対し過敏症の既往歴のある患者 昏睡状態の患者 バルビツール酸誘導体等の中枢神経抑制...,0,NO_SUBTITLE,-84.765291
3466,63449,オーストラリア - 眼瞼痙攣、片側顔面痙攣、第7脳神経障害（12歳以上）、小児脳性麻痺患者に...,1,概要,-82.381263
5445,12437,日本の法制度上では、過去にメタノールが主成分のガイアックスを高濃度アルコール燃料と名指しした...,1,利用 自動車燃料,-79.998899
1700,3208855,脳血管障害、脳性麻痺、痙性脊髄麻痺、脊髄血管障害、 頸部脊椎症、後縦靱帯骨化症 多発性硬化症...,1,適応症 バクロフェン髄腔内持続投与療法,-76.727534
235,3460340,重大な副作用としては、 中毒性表皮壊死症（0.1％）、皮膚粘膜眼症候群（0.7％）、過敏症症...,0,NO_SUBTITLE,-70.574538
114,33814,金沢大学医学部整形外科では、独自に開発したカフェイン併用療法を抗がん剤治療の術前に用いて、骨...,0,医薬品として 高度先進医療,-68.045041


In [174]:
tmp_df.to_csv("../dump/tmp3.csv")

## 害に頻出する手がかり語を抽出

In [175]:
Youto_sentence_df[Youto_sentence_df.sentence.str.contains('副作用|有害|毒性|危険|禁忌')]

,sentence,_id
88,一部の炎症性疾患（中等度のアレルギー反応等）の治療に用いられるほか、高用量で癌の治療に用いら...,1834328
91,フェナセチン（英:Phenacetin）は、かつて広く使用されていた鎮痛剤であるが、その副作...,2306663
149,工業的にもよく用いられる溶媒であるが、人体に有害であり、吸入や皮膚の接触は避けなければならない。,770695
153,粘膜組織をわずかに刺激するものの毒性は非常に低いため、石鹸やシャンプーに多く用いられる。,506816
164,TBHP の利用は、塩基を作用させ t-BuOO- の形で求核剤とする利用と、毒性や価格が高...,1439659
185,エリトリトールには定期的な使用における副作用はないが、大量のエリトリトールの摂取によって下剤...,696877
186,また、細胞毒性に着目して使用された最初の抗がん剤であり、白血病や悪性リンパ腫の治療薬として使...,633651
205,しかし急性心不全や肺水腫などの重篤な副作用が発生し、死亡に至る例もあったため、日本脳卒中学会...,1136476
211,NSAIDs投与に起因する消化性潰瘍の予防効果は、少量のプロトンポンプ阻害薬（以下：PPI）...,1186509
213,医療資源の乏しい地域では、出産時にミソプロストールを無作為に使用した群と使用しなかった対照群...,1186509


In [176]:
m = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
clue_words = []
for i, v in Youto_sentence_df.iterrows():
    node = m.parseToNode(v.sentence)
    clue_word = []
    while node:
        if len(node.surface) is 0:
            node = node.next
            continue
        
        hinshi = node.feature.split(',')
        if hinshi[0] == '名詞':
            clue_word.append(node.surface)
            
        node = node.next
        
    clue_words.append(clue_word)

In [177]:
side_effect_df = \
pd.DataFrame(
    {
        "clue_word": clue_words
         , "label": Youto_sentence_df.sentence.str.contains('副作用|有害|毒性|危険|禁忌')
    }
)

In [154]:
def word_proba(docs, word):
    word_count = np.array([Counter(doc)[word] for doc in docs])
    return pd.Series(word_count / np.sum(word_count))

def word_entropy(word_proba):
    return np.sum(np.nan_to_num(-word_proba * np.log2(word_proba)))

In [178]:
word_set = set(flatten(side_effect_df.clue_word.values))

word_entropy_positive = \
    np.array([side_effect_df.loc[side_effect_df.label == 1, 'clue_word'].pipe(word_proba, word).pipe(word_entropy) for word in word_set]) \
    / np.log2(len(side_effect_df.loc[side_effect_df.label == 1]))
word_entropy_negative = \
    np.array([side_effect_df.loc[side_effect_df.label == 0, 'clue_word'].pipe(word_proba, word).pipe(word_entropy) for word in word_set]) \
    / np.log2(len(side_effect_df.loc[side_effect_df.label == 0]))

/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log2
  


In [179]:
word_entropy_df = pd.DataFrame({"clue_word": list(word_set), "entropy_positive": word_entropy_positive, "entropy_negative": word_entropy_negative})
word_entropy_df.head()

,clue_word,entropy_positive,entropy_negative
0,楽,0.000000,0.000000
1,TNT,0.000000,0.000000
2,アメリカ合衆国,0.250533,0.388412
3,マイクロエレクトロニクス,0.000000,0.000000
4,ジヒドロキシアセトンリン酸,0.000000,0.000000


In [196]:
alpha = 2.
extract_clue_word_df = \
    word_entropy_df[
        (word_entropy_df.entropy_positive > (alpha * word_entropy_df.entropy_negative))
        & (word_entropy_df.clue_word.str.contains('副作用|有害|毒性|危険|禁忌')
           | word_entropy_df.entropy_negative > 0)
    ]
extract_clue_word_df

,clue_word,entropy_positive,entropy_negative
111,一定,0.198542,0.080795
546,使用制限,0.198542,0.080795
1770,房,0.171734,0.080795
2126,有害性,0.323809,0.000000
2154,ポリビニルアルコール,0.198542,0.080795
2206,黄疸,0.198542,0.080795
2519,症例,0.169364,0.080795
3513,口,0.250533,0.080795
3849,角膜炎,0.198542,0.080795
5096,心停止,0.198542,0.080795


In [190]:
word_entropy_df[word_entropy_df.clue_word == '危険']

,clue_word,entropy_positive,entropy_negative
10977,危険,0.550209,0.0
